<a href="https://colab.research.google.com/github/hamidzangiabadi/sailfish-optimization-algorithm/blob/main/Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install ucimlrepo
# !pip install mealpy
# !pip install scikit-learn
!pip install uci-dataset

# import datasets

In [1]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
# iris = fetch_ucirepo(id=53) 
# breast_cancer = fetch_ucirepo(id=14)
# communities_and_crime = fetch_ucirepo(id=183) 
# zoo = fetch_ucirepo(id=111)
# automobile = fetch_ucirepo(id=10)
# print("imported")
# lymphography = fetch_ucirepo(id=63)
# print("imported")
# student_performance = fetch_ucirepo(id=320) 
# ionosphere = fetch_ucirepo(id=52)
# credit_approval = fetch_ucirepo(id=27) 
# hepatitis = fetch_ucirepo(id=46) 
# letter_recognition = fetch_ucirepo(id=59)
# heart_disease = fetch_ucirepo(id=45) 
tic_tac_toe_endgame = fetch_ucirepo(id=101) 

print("imported")


imported


In [2]:
import pandas as pd


Datasets = [
#             { "name" : "Iris" , "dataset" : iris },
#              { "name" : "Breast Cancer" , "dataset" : breast_cancer },
#              { "name" : "Zoo" , "dataset" : zoo },
#              { "name" : "Automobile" , "dataset" : automobile },
#              { "name" : "Lymphography" , "dataset" : lymphography },
#              { "name" : "Student Performance" , "dataset" : student_performance },
#              { "name" : "Ionosphere" , "dataset" : ionosphere },
#              { "name" : "Credit Approval" , "dataset" : credit_approval },
#              { "name" : "Hepatitis" , "dataset" : hepatitis },
#                { "name" : "Communities and Crime" , "dataset" : communities_and_crime },
#                 { "name" : "Letter Recognition" , "dataset" : letter_recognition },
#     { "name" : "Heart Disease" , "dataset" : heart_disease },
    { "name" : "Tic-Tac-Toe Endgame" , "dataset" : tic_tac_toe_endgame },
           ]


dataset = tic_tac_toe_endgame

## modify datasets and required information





Preprocessing to convert non int and float columns using label encoder

In [3]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

def encode_categorical_features(df):
    label_encoder = LabelEncoder()
    object_columns = df.select_dtypes(include=['object']).columns
    
    for column in object_columns:
        df[column + '_encoded'] = label_encoder.fit_transform(df[column])
    
    df.drop(columns=object_columns, inplace=True)

def preprocess_data(dataset):
    encode_categorical_features(dataset)
    encode_categorical_features(dataset)


preprocess_data(dataset.data.features)



data = pd.concat([dataset.data.features, dataset.data.targets], axis=1)
data.dropna(inplace=True)
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]
dataset.data.targets = y
dataset.data.features = X


## functions

In [4]:
import csv

def binary_conversion(X, dim):
    Xbin = np.zeros(dim)
    for d in range(dim):
        if X[d] > 0.5:
            Xbin[d] = 1
        else:
            Xbin[d] = 0

    return Xbin


def save_results(g_best, method_name, num_features,datasetName):
    
    selectedFeaturesSubset = binary_conversion(g_best.solution,num_features)
    featuresCount = int(sum(selectedFeaturesSubset))
    bestFitness = g_best.target.fitness
    
    print(f"Solution: {str(selectedFeaturesSubset)}, Fitness: {g_best.target.fitness}")
    print(f"No of features: {str(featuresCount)}")
    print(f"Reached accuracy: {str(checkAccuracy(selectedFeaturesSubset))}");
    
    acc = checkAccuracy(selectedFeaturesSubset)
    res = {
        "Method": method_name,
        "NumberOfFeatures": str(featuresCount),
        "Accuracy": acc,
        "Error":1-acc,
        "Fitness": bestFitness
    }
    # Save to CSV or any other preferred format
    # Example: Using pandas to save as CSV
    import pandas as pd
    df = pd.DataFrame([res])
    df.to_csv(f"{datasetName}_results.csv", mode='a', index=False, header=not pd.io.common.file_exists(f"{datasetName}_results.csv"))
    

## method

In [ ]:
import numpy as np
from mealpy import FloatVar, GA,SFO,WOA,EHO,GWO,DO,MVO,MFO
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

X_train, X_test, y_train, y_test = "", "" ,"" ,""



def checkAccuracy(features):
    max_feat = len(features)
    num_feat = np.sum(binary_conversion(features,num_features) == 1)
    if num_feat == 0 or num_feat == max_feat:
        return 1  

    targets = np.array(y).ravel()
    selected_features = binary_conversion(features,num_features);
    f =  X.iloc[:, selected_features == 1]
    X_train, X_test, y_train, y_test = train_test_split(f, targets, test_size=0.3)
    svm = SVC(kernel='linear')
    svm.fit(X_train, y_train.ravel())
    # Predict on test set
    y_pred = svm.predict(X_test)
    Ytest = np.array(y_test)
    num_valid = np.size(X_test, 0)
    yvalid  = Ytest.reshape(num_valid)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(accuracy)
    return accuracy

def objective_function(features, X_train, X_test, y_train, y_test):
    max_feat = len(features)
    num_feat = np.sum(binary_conversion(features, num_features) == 1)
    
    # If no features or all features are selected, return a poor score
    if num_feat == 0 or num_feat == max_feat:
        return float('inf')  # Adjusted to return a poor score

    alpha = 0.6
    beta = 0.4

    selected_features = binary_conversion(features, X_train.shape[1])
    f_train = X_train.iloc[:, selected_features == 1]
    f_test = X_test.iloc[:, selected_features == 1]
    
    svm = SVC(kernel='linear')
    svm.fit(f_train, np.array(y_train).ravel())
    y_pred = svm.predict(f_test)

    accuracy = accuracy_score(y_test, y_pred)
    cost = (alpha * (1 - accuracy)) + (beta * (num_feat / max_feat))
    return cost



for dataset in Datasets:
    X = dataset["dataset"].data.features
    y = dataset["dataset"].data.targets

    num_features = X.shape[1]
    search_space = np.array([0, 1] * num_features)

    lb = np.zeros(num_features)
    ub = np.ones(num_features)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
   
    
    problem_dict = {
    "bounds": FloatVar(lb=lb, ub=ub, ),
    "obj_func": lambda features: objective_function(features, X_train, X_test, y_train, y_test),
    "minmax": "min",
    }
    
    for i in range(10):
    
        

    
        SFOModel = SFO.OriginalSFO(epoch=30,pop_size=50, pp = 0.1, AP =4 ,epsilon = 0.01)
        g_best = SFOModel.solve(problem_dict)
        save_results(g_best,"SFO", num_features, dataset["name"])
        
        
        GAmodel = GA.BaseGA(epoch=30,pop_size=50,pc=0.7, pm=0.05)
        g_best = GAmodel.solve(problem_dict)
        save_results(g_best,"GA", num_features, dataset["name"])
        
        EHOModel = EHO.OriginalEHO(epoch=30,pop_size=50, alpha = 0.5, beta = 0.5, n_clans = 2)
        g_best = EHOModel.solve(problem_dict)
        save_results(g_best,"EHO", num_features, dataset["name"])

        GWOModel = GWO.OriginalGWO(epoch=30,pop_size=50)
        g_best = GWOModel.solve(problem_dict)
        save_results(g_best,"GWO", num_features, dataset["name"])

        WOAModel = WOA.OriginalWOA(epoch=30,pop_size=50)
        g_best = WOAModel.solve(problem_dict)
        save_results(g_best,"WOA", num_features, dataset["name"])

        MFOModel = MFO.OriginalMFO(epoch=30,pop_size=50)
        g_best = MFOModel.solve(problem_dict)
        save_results(g_best,"MFO", num_features, dataset["name"])


2024/05/19 12:34:04 AM, INFO, mealpy.swarm_based.SFO.OriginalSFO: Solving single objective optimization problem.
2024/05/19 12:34:13 AM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Problem: P, Epoch: 1, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.87443 seconds
2024/05/19 12:34:14 AM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Problem: P, Epoch: 2, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.84670 seconds
2024/05/19 12:34:14 AM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Problem: P, Epoch: 3, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.78749 seconds
2024/05/19 12:34:15 AM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Problem: P, Epoch: 4, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 1.07180 seconds
2024/05/19 12:34:16 AM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Problem: P, Epoch: 5, Current best: 0.24444444444444446, Global best: 0.244444

Solution: [0. 0. 0. 0. 0. 0. 1. 0. 0.], Fitness: 0.24444444444444446
No of features: 1
0.6284722222222222
Reached accuracy: 0.6284722222222222
0.6875


2024/05/19 12:34:39 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 1, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.61915 seconds
2024/05/19 12:34:39 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 2, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.57293 seconds
2024/05/19 12:34:40 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 3, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.55090 seconds
2024/05/19 12:34:41 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 4, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.59058 seconds
2024/05/19 12:34:41 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 5, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.61648 seconds
2024/05/19 12:34:42 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>

Solution: [0. 0. 1. 0. 0. 0. 0. 0. 0.], Fitness: 0.24444444444444446
No of features: 1
0.6597222222222222
Reached accuracy: 0.6597222222222222
0.6597222222222222


2024/05/19 12:34:57 AM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoch: 1, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.76035 seconds
2024/05/19 12:34:58 AM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoch: 2, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.72163 seconds
2024/05/19 12:34:59 AM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoch: 3, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.71807 seconds
2024/05/19 12:35:00 AM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoch: 4, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.70306 seconds
2024/05/19 12:35:00 AM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoch: 5, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.68159 seconds
2024/05/19 12:35:01 AM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Proble

...

2024/05/19 12:53:26 AM, INFO, mealpy.swarm_based.MFO.OriginalMFO: >>>Problem: P, Epoch: 1, Current best: 0.2888888888888889, Global best: 0.2888888888888889, Runtime: 0.73355 seconds
2024/05/19 12:53:27 AM, INFO, mealpy.swarm_based.MFO.OriginalMFO: >>>Problem: P, Epoch: 2, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.72254 seconds
2024/05/19 12:53:27 AM, INFO, mealpy.swarm_based.MFO.OriginalMFO: >>>Problem: P, Epoch: 3, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.72118 seconds
2024/05/19 12:53:28 AM, INFO, mealpy.swarm_based.MFO.OriginalMFO: >>>Problem: P, Epoch: 4, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.66650 seconds
2024/05/19 12:53:29 AM, INFO, mealpy.swarm_based.MFO.OriginalMFO: >>>Problem: P, Epoch: 5, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.67442 seconds
2024/05/19 12:53:29 AM, INFO, mealpy.swarm_based.MFO.OriginalMFO: >>>Problem:

Solution: [0. 1. 0. 0. 0. 0. 0. 0. 0.], Fitness: 0.24444444444444446
No of features: 1
0.6284722222222222
Reached accuracy: 0.6284722222222222
0.6666666666666666


2024/05/19 12:53:53 AM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Problem: P, Epoch: 1, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 1.05393 seconds
2024/05/19 12:53:54 AM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Problem: P, Epoch: 2, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.97848 seconds
2024/05/19 12:53:55 AM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Problem: P, Epoch: 3, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.90993 seconds
2024/05/19 12:53:56 AM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Problem: P, Epoch: 4, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.96841 seconds
2024/05/19 12:53:57 AM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Problem: P, Epoch: 5, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 1.04909 seconds
2024/05/19 12:53:58 AM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Proble

Solution: [0. 1. 0. 0. 0. 0. 0. 0. 0.], Fitness: 0.24444444444444446
No of features: 1
0.6388888888888888
Reached accuracy: 0.6388888888888888
0.6527777777777778


2024/05/19 12:54:22 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 1, Current best: 0.2888888888888889, Global best: 0.2888888888888889, Runtime: 0.72468 seconds
2024/05/19 12:54:23 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 2, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.65968 seconds
2024/05/19 12:54:23 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 3, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.56627 seconds
2024/05/19 12:54:24 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 4, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.53782 seconds
2024/05/19 12:54:24 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 5, Current best: 0.24444444444444446, Global best: 0.24444444444444446, Runtime: 0.55820 seconds
2024/05/19 12:54:25 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Pr

# create plots

In [ ]:

import numpy as np
import matplotlib.pyplot as plt



# set width of bar
barWidth = 0.25
fig = plt.subplots(figsize =(12, 8))


SFO = [];
GA = [];

DatasetsName = []


for x in Datasets:
  SFO.append(float(x["Accuracy_SFO"]) * 100)
  GA.append(float(x["Accuracy_GA"]) * 100)
  DatasetsName.append(x["name"])


# Set position of bar on X axis
br1 = np.arange(len(SFO))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]

# Make the plot
plt.bar(br1, SFO, color ='r', width = barWidth,
        edgecolor ='grey', label ='SFO')
plt.bar(br2, GA, color ='g', width = barWidth,
        edgecolor ='grey', label ='GA')

# Adding Xticks
plt.xlabel('Branch', fontweight ='bold', fontsize = 15)
plt.ylabel('Students passed', fontweight ='bold', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(SFO))],
       DatasetsName)

plt.legend()
plt.show()